## Accessing Landsat Collection 2 Level-1 and Level-2 data with the Planetary Computer STAC API

The Planetary Computer's [Landsat](https://landsat.gsfc.nasa.gov/) dataset represents a global archive of [Level-1](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-1-data) and [Level-2](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products) data from from [Landsat Collection 2](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2). The dataset is grouped into two STAC Collections:
- `landsat-c2-l2` for Level-2 data
- `landsat-c2-l1` for Level-1 data

This notebook demonstrates the use of the Planetary Computer STAC API to query for Landsat data. We will start with an example using Level-2 data and follow with a similar example using Level-1 data.

### Environment setup

This notebook works with or without an API key, but you will be given more permissive access to the data with an API key. 
- The [Planetary Computer Hub](https://planetarycomputer.microsoft.com/compute) is pre-configured to use your API key.
- To use your API key locally, set the environment variable `PC_SDK_SUBSCRIPTION_KEY` or use `pc.settings.set_subscription_key(<YOUR API Key>)`.

In [1]:
import pystac_client
import planetary_computer
import odc.stac
import matplotlib.pyplot as plt
import pandas as pd
import csv
from datetime import datetime
from pystac.extensions.eo import EOExtension as eo
import numpy as np

### Data access

The datasets hosted by the Planetary Computer are available from [Azure Blob Storage](https://docs.microsoft.com/en-us/azure/storage/blobs/). We'll use [pystac-client](https://pystac-client.readthedocs.io/) to search the Planetary Computer's [STAC API](https://planetarycomputer.microsoft.com/api/stac/v1/docs) for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage. We'll specify a `modifier` so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See [Reading from the STAC API](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) and [Using tokens for data access](https://planetarycomputer.microsoft.com/docs/concepts/sas/) for more.

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

### Choose an area and time of interest

This area is in Redmond, WA, USA, near Microsoft's main campus. We'll search all of 2021, limiting the results to those less than 10% cloudy.

In [3]:
time = '2022-01-01/2022-08-31'

In [4]:
bbox = [105.248554, 10.510542, 105.248554, 10.510542]

In [5]:
bbox_of_interest = [-122.2751, 47.5469, -121.9613, 47.7458]
time_of_interest = "2021-01-01/2021-12-31"

In [6]:
search = catalog.search(
    collections=["landsat-c2-l2"],
    bbox=bbox,
    datetime=time,
    query={"eo:cloud_cover": {"lt": 10}},
)

items = search.item_collection()
print(f"Returned {len(items)} Items")

for item in items:
    date = item.properties['datetime']
    print(date)

Returned 4 Items
2022-02-27T03:20:28.774876Z
2022-02-19T03:20:27.771564Z
2022-01-18T03:20:37.922626Z
2022-01-02T03:20:40.283448Z


Let's find the least cloudy of the bunch.

In [7]:
selected_item = min(items, key=lambda item: eo.ext(item).cloud_cover)

print(
    f"Choosing {selected_item.id} from {selected_item.datetime.date()}"
    + f" with {selected_item.properties['eo:cloud_cover']}% cloud cover"
)

Choosing LC08_L2SP_126053_20220102_02_T1 from 2022-01-02 with 1.43% cloud cover


### Available assets

In addition to numerous metadata assets, each Electro-Optical (EO) band is a separate asset.

In [8]:
max_key_length = len(max(selected_item.assets, key=len))
for key, asset in selected_item.assets.items():
    print(f"{key.rjust(max_key_length)}: {asset.title}")

              qa: Surface Temperature Quality Assessment Band
             ang: Angle Coefficients File
             red: Red Band
            blue: Blue Band
            drad: Downwelled Radiance Band
            emis: Emissivity Band
            emsd: Emissivity Standard Deviation Band
            trad: Thermal Radiance Band
            urad: Upwelled Radiance Band
           atran: Atmospheric Transmittance Band
           cdist: Cloud Distance Band
           green: Green Band
           nir08: Near Infrared Band 0.8
          lwir11: Surface Temperature Band
          swir16: Short-wave Infrared Band 1.6
          swir22: Short-wave Infrared Band 2.2
         coastal: Coastal/Aerosol Band
         mtl.txt: Product Metadata File (txt)
         mtl.xml: Product Metadata File (xml)
        mtl.json: Product Metadata File (json)
        qa_pixel: Pixel Quality Assessment Band
       qa_radsat: Radiometric Saturation and Terrain Occlusion Quality Assessment Band
      qa_aerosol: Aeros

### Render a natural color image of the AOI

We'll start by loading the red, green, and blue bands for our area of interest into an xarray dataset using [`odc-stac`](https://pypi.org/project/odc-stac/). We will also load the nir08 band for use in computing an NDVI value in a later example. Note that we pass the `sign` function from the [planetary-computer](https://github.com/microsoft/planetary-computer-sdk-for-python) package so that `odc-stac` can supply the required [Shared Access Signature](https://docs.microsoft.com/en-us/azure/storage/common/storage-sas-overview) tokens that are necessary to download the asset data.

In [9]:
bands_of_interest = ["nir08", "red", "green", "blue", "qa_pixel", "lwir11"]
data = odc.stac.stac_load(
    [selected_item], bands=bands_of_interest, bbox=bbox
).isel(time=0)
data

<xarray.Dataset>
Dimensions:      (y: 1, x: 1)
Coordinates:
  * y            (y) float64 1.162e+06
  * x            (x) float64 5.272e+05
    spatial_ref  int32 32648
    time         datetime64[ns] 2022-01-02T03:20:40.283448
Data variables:
    nir08        (y, x) uint16 12103
    red          (y, x) uint16 10332
    green        (y, x) uint16 9986
    blue         (y, x) uint16 9143
    qa_pixel     (y, x) uint16 21824
    lwir11       (y, x) uint16 45670

Now we'll convert our xarray Dataset to a DataArray and plot the RGB image. We set `robust=True` in `imshow` to avoid manual computation of the color limits (vmin and vmax) that is necessary for data not scaled to between 0-1 while also eliminating extreme values that can cause a washed out image.

In [10]:
from tqdm import tqdm
tqdm.pandas()
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [11]:
df = pd.read_csv('Crop_Yield_Data_challenge_2.csv')

In [12]:
import xarray as xr
import pandas as pd

# Load the data into an xarray Dataset
data = odc.stac.stac_load(
    [selected_item],
    bands=bands_of_interest,
    bbox=bbox
)

# Create an empty dataframe to hold the merged data
merged_df = pd.DataFrame()
dataframes = {}

# Concatenate the dataframes horizontally
for var_name in data.data_vars:
    var_data = data[var_name].to_dataframe().reset_index()
    var_data = var_data.loc[:, ~var_data.columns.duplicated()]
    merged_df = pd.concat([merged_df, var_data], axis=1)
    dataframes[var_name] = var_data

# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_data.csv', index=False)

# Access the dataframes
red_df = dataframes['red']
green_df = dataframes['green']
blue_df = dataframes['blue']
nir_df = dataframes['nir08']
qa_pixel_df = dataframes['qa_pixel']
lwir11_df = dataframes['lwir11']

# Save each dataframe as a CSV file
red_df.to_csv('red_data.csv', index=False)
green_df.to_csv('green_data.csv', index=False)
blue_df.to_csv('blue_data.csv', index=False)
nir_df.to_csv('nir_data.csv', index=False)
qa_pixel_df.to_csv('qa_pixel_data.csv', index=False)
lwir11_df.to_csv('lwir11_data.csv', index=False)
print(red_df['red'])

0    10332
Name: red, dtype: uint16


In [13]:
def get_ndvi(longitude, latitude, season, date, csv_file):
    #time = '2022-01-01/2022-08-31'
    str_latitude = str(latitude)
    found_start_date = False
    if season == "SA":
        with open(csv_file, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if (row['Latitude'] == str_latitude) & (row['Season(SA = Summer Autumn, WS = Winter Spring)'] != season):
                    start_date = row['Date of Harvest']
                    found_start_date = True
                    #print(start_date)

        date_obj = datetime.strptime(date, "%d-%m-%Y")
        new_date_csv = date_obj.strftime("%Y-%m-%d")

        if found_start_date:
            #print("Found")
            start_date_obj = datetime.strptime(start_date, "%d-%m-%Y")
            new_start_date_csv = start_date_obj.strftime("%Y-%m-%d")
            if new_start_date_csv > "2022-04-01":
                #print("bigger")
                datetime1 = new_start_date_csv + "/" + new_date_csv
                #print(datetime1)

            else:
                #print("smaller")
                datetime1 = "2022-04-01" + "/" + new_date_csv
                #print(datetime1)

        else:
            #print("not")
            datetime1 = "2022-04-01" + "/" + new_date_csv
            #print(datetime1)


    if season == 'WS':
        date_obj = datetime.strptime(date, "%d-%m-%Y")
        new_date_csv = date_obj.strftime("%Y-%m-%d")
        datetime1 = "2021-11-01" +"/" + new_date_csv
        #print(datetime1)
    bbox = [longitude, latitude, longitude, latitude]
    search = catalog.search(
        collections=["landsat-c2-l2"],
        bbox=bbox,
        datetime=time,
        query={"eo:cloud_cover": {"lt": 10}}, # 10% cloud coverage
    )

    items = search.item_collection()
    if items:
        selected_item = min(items, key=lambda item: eo.ext(item).cloud_cover)
        #selected_item = min(items, key=lambda item: eo.ext(item).cloud_cover)
        bands_of_interest = ["nir08", "red", "green", "blue", "qa_pixel", "lwir11"]
        data = odc.stac.stac_load([selected_item], bands=bands_of_interest, bbox=bbox).isel(time=0)

        #df = pd.read_csv('Crop_Yield_Data_challenge_2.csv')
        dataframes = {}
        
        # Concatenate the dataframes horizontally
        for var_name in data.data_vars:
            var_data = data[var_name].to_dataframe().reset_index()
            var_data = var_data.loc[:, ~var_data.columns.duplicated()]
            dataframes[var_name] = var_data
        red_df = dataframes['red']
        green_df = dataframes['green']
        blue_df = dataframes['blue']
        nir_df = dataframes['nir08']
        qa_pixel_df = dataframes['qa_pixel']
        lwir11_df = dataframes['lwir11']
        
        return red_df['red'], nir_df['nir08']
    else:
        selected_item = "N/A"
        return np.nan, np.nan

    '''
    selected_item = min(items, key=lambda item: eo.ext(item).cloud_cover)
    bands_of_interest = ["nir08", "red", "green", "blue", "qa_pixel", "lwir11"]
    data = odc.stac.stac_load([selected_item], bands=bands_of_interest, bbox=bbox).isel(time=0)
    
    #df = pd.read_csv('Crop_Yield_Data_challenge_2.csv')
    dataframes = {}

    # Concatenate the dataframes horizontally
    for var_name in data.data_vars:
        var_data = data[var_name].to_dataframe().reset_index()
        var_data = var_data.loc[:, ~var_data.columns.duplicated()]
        dataframes[var_name] = var_data
    red_df = dataframes['red']
    green_df = dataframes['green']
    blue_df = dataframes['blue']
    nir_df = dataframes['nir08']
    qa_pixel_df = dataframes['qa_pixel']
    lwir11_df = dataframes['lwir11']
    '''
    
    

In [14]:
new_columns = df.progress_apply(lambda x: get_ndvi(x['Longitude'], x['Latitude'],x['Season(SA = Summer Autumn, WS = Winter Spring)'], x['Date of Harvest'], 'Crop_Yield_Data_challenge_2.csv'), axis=1)


  0%|          | 0/557 [00:00<?, ?it/s]

In [15]:
print(new_columns)

0      ([10332], [12103])
1       ([9857], [10104])
2       ([9102], [13276])
3       ([8458], [10473])
4       ([9031], [21966])
              ...        
552    ([10677], [12468])
553     ([9476], [11019])
554     ([8903], [10145])
555     ([8844], [15231])
556     ([9218], [12030])
Length: 557, dtype: object


In [16]:
print(new_columns[0][1][0])

12103


In [22]:
ndvi_values = []  # Create an empty list to store NDVI values

for i in new_columns:
    if isinstance(i[0], float) or isinstance(i[1], float):
        ndvi = np.nan
    elif i[0].isnull().any() or i[1].isnull().any():
        ndvi = np.nan
    else:
        red = i[0][0]
        nir = i[1][0]
        ndvi = (nir - red) / (nir + red)
    
    ndvi_values.append(ndvi)  # Append the calculated NDVI value to the list

new = pd.DataFrame({'ndvi': ndvi_values})
print(new)


         ndvi
0    0.078939
1    0.012374
2    0.186522
3    0.106439
4    0.417298
..        ...
552  0.077382
553  0.075287
554  0.065204
555  0.265296
556  0.132342

[557 rows x 1 columns]


/tmp/ipykernel_615/2100125459.py:11: RuntimeWarning: overflow encountered in ushort_scalars
  ndvi = (nir - red) / (nir + red)


In [24]:
new.to_csv("new_crop_data.csv", index=False, header=["ndvi"])


In [ ]:

'''
red = data["red"].astype("float")
nir = data["nir08"].astype("float")
ndvi = (nir - red) / (nir + red)
'''

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

data[["red", "green", "blue"]].to_array().plot.imshow(robust=True, ax=ax)
ax.set_title("Natural Color, Redmond, WA");

### Render an NDVI image of the AOI

Landsat has several bands, and with them we can go beyond rendering natural color imagery; for example, the following code computes a [Normalized Difference Vegetation Index (NDVI)](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) using the near-infrared and red bands. Note that we convert the red and near infrared bands to a data type that can contain negative values; if this is not done, negative NDVI values will be incorrectly stored.

In [ ]:
data[["red", "green", "blue"]].to_array().plot.imshow(robust=True, ax=ax)
ax.set_title("Natural Color, Redmond, WA");
red = data["red"].astype("float")
nir = data["nir08"].astype("float")
ndvi = (nir - red) / (nir + red)

fig, ax = plt.subplots(figsize=(14, 10))
ndvi.plot.imshow(ax=ax, cmap="viridis")
ax.set_title("NDVI, Redmond, WA");

### Selecting specific platforms

Landsat Collection 2 Level-2 includes assets from several different Platforms.

In [ ]:
catalog.get_collection("landsat-c2-l2").summaries.to_dict()["platform"]

In [ ]:
search = catalog.search(
    collections=["landsat-c2-l2"],
    bbox=bbox_of_interest,
    datetime=time_of_interest,
    query={
        "eo:cloud_cover": {"lt": 10},
        "platform": {"in": ["landsat-8", "landsat-9"]},
    },
)
items = search.get_all_items()

### Rescaling Temperature Data

Landsat Collection 2 Level 2 includes measures of [Surface Temperature](https://www.usgs.gov/landsat-missions/landsat-surface-temperature). We'll look at the surface temperature band `TIRS_B10` under the key `lwir11`. The raw values are rescaled, so you should scale and offset the data before interpreting it. Use the metadata in the asset's `raster_bands` to find the scale and offset values:

In [ ]:
band_info = selected_item.assets["lwir11"].extra_fields["raster:bands"][0]
band_info

To go from the raw values to Kelvin, we apply those values:

In [ ]:
temperature = data["lwir11"].astype(float)
temperature *= band_info["scale"]
temperature += band_info["offset"]
temperature[:5, :5]

To convert from Kelvin to degrees, subtract 273.15.

In [ ]:
celsius = temperature - 273.15
celsius.plot(cmap="magma", size=10);

### Landsat Collection 2 Level-1 Data

Thus far we have worked with Landsat Collection 2 Level-2 data, which is processed to a consistent set of surface reflectance and surface temperature [science products](https://www.usgs.gov/landsat-missions/landsat-science-products). 

The Planetary Computer also hosts Collection 2 Level-1 data (top of atmosphere values) acquired by the [Multispectral Scanner System](https://landsat.gsfc.nasa.gov/multispectral-scanner-system/) (MSS) onboard Landsat 1 through 5. These data do not include a blue band, so a natural color image is not possible. We will plot a color infrared image from the nir08, red, and green bands instead.

As before, we use pystac-client to search over the `landsat-c2-l1` collection. We'll use the same area of interest and return only those with less than 10% cloud cover.

In [ ]:
search = catalog.search(
    collections=["landsat-c2-l1"],
    bbox=bbox,
    query={"eo:cloud_cover": {"lt": 10}},
)

items = search.item_collection()
print(f"Returned {len(items)} Items")
item = items[0]
for item in items:
    date = item.properties["datetime"]
    print(date)

Choose the least cloudy Item.

In [ ]:
selected_item = min(items, key=lambda item: eo.ext(item).cloud_cover)

print(
    f"Choosing {selected_item.id} from {selected_item.datetime.date()}"
    + f" with {selected_item.properties['eo:cloud_cover']}% cloud cover"
)

### Available assets

MSS data has four EO bands assets and a number of metadata files and bands.

In [ ]:
max_key_length = len(max(selected_item.assets, key=len))
for key, asset in selected_item.assets.items():
    print(f"{key.rjust(max_key_length)}: {asset.title}")

We'll use `odc-stac` to load only the EO bands and area we are interested in.

In [ ]:
bands_of_interest = ["nir08", "red", "green"]
data = odc.stac.stac_load(
    [selected_item], bands=bands_of_interest, bbox=bbox
).isel(time=0)

cir = data.to_array()

fig, ax = plt.subplots(figsize=(10, 10))
cir.plot.imshow(robust=True, ax=ax)
ax.set_title("Color Infrared, Redmond, WA");

In [ ]:
cir